In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    BartForConditionalGeneration,
)
import collections
import numpy as np
from transformers.data.data_collator import default_data_collator
from indobenchmark import IndoNLGTokenizer

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#kalau t5
MODEL_CHECKPOINT = "../models/pt-indot5-TA_PT"
TOK_CHECKPOINT = "Wikidepia/IndoT5-base"
SAVE_PATH = "../models/pt-indot5-MLM_TA_PT"
#kalau bart
# MODEL_CHECKPOINT = "indobenchmark/indobart-v2"
# SAVE_PATH = "models/pt-indobart-MLM_PT"

In [4]:
# Set up the t5 tokenizer
tokenizer = AutoTokenizer.from_pretrained(TOK_CHECKPOINT)

#set up bart tokenizer
#tokenizer = IndoNLGTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Data

In [5]:
CHUNK_SIZE = 128

In [6]:
def tokenize_function(examples):
    result = tokenizer(examples["clean_tweet"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [7]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // CHUNK_SIZE) * CHUNK_SIZE
    # Split by chunks of max_len
    result = {
        k: [t[i : i + CHUNK_SIZE] for i in range(0, total_length, CHUNK_SIZE)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
def drop_duplicate_id(tokens, highest_id):
    extra_ids = tokenizer.convert_tokens_to_ids([f'<extra_id_{id}>' for id in range(highest_id)])
    extra_ids = {key:0 for key in extra_ids}
    del_idx = []
    for i,tok in enumerate(tokens):
        if tok in extra_ids:
            extra_ids[tok]+=1
            if extra_ids[tok]>1:
                del_idx.append(i)
    new_tokens = [tokens[i] for i in range(len(tokens)) if i not in del_idx]
    return new_tokens

In [9]:
wwm_probability = 0.2


def multi_word_masking(examples):
    new_inputs = []
    new_labels = []
    for i in range(len(examples['input_ids'])):
        word_ids = examples['word_ids'][i]
        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = examples["input_ids"][i]
        labels = examples["labels"][i]
        input_masked = -1
        label_masked = -1
        prev_mask = None
        for word_id, masked in enumerate(mask):
            if prev_mask!=masked:
                if masked==0:
                    label_masked+=1
                elif masked==1:
                    input_masked+=1
                prev_mask=masked
            for idx in mapping[word_id]:
                #if not masking input_ids then we mask the label
                if masked==0:
                    labels[idx] = tokenizer.convert_tokens_to_ids(f'<extra_id_{label_masked}>')
                #if masking then we mask the input_ids
                elif masked==1:
                    input_ids[idx] = tokenizer.convert_tokens_to_ids(f'<extra_id_{input_masked}>')
        #dropping the same extra_id
        input_ids = drop_duplicate_id(input_ids, input_masked+1)
        labels = drop_duplicate_id(labels, label_masked+1)
        new_inputs.append(input_ids)
        new_labels.append(labels)
    examples['labels'] = new_labels
    examples['input_ids'] = new_inputs
    examples['attention_mask'] = [[1 for i in range(len(examples['input_ids'][j]))] for j in range(len(examples['input_ids']))]
    return examples

In [10]:
raw_dataset = load_dataset('csv', data_files='../Data/post-train/MLM/clean_tweet.csv')
tokenized_datasets = raw_dataset.map(
    tokenize_function, batched=True, remove_columns=raw_dataset['train'].column_names
)
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
downsampled_dataset = lm_datasets["train"].train_test_split(test_size=0.1, seed=42)
downsampled_dataset = downsampled_dataset.map(multi_word_masking, batched=True)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-a0da231f93618037/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 34.50it/s]
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-a0da231f93618037\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ecfd2fdf1dcb97c8.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-a0da231f93618037\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c68c1a0da8738c95.arrow
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-a0da231f93618037\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-8e9f8a7eceb1a98c.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-a0da231f93618037\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bc

In [11]:
tokenizer.decode(downsampled_dataset['train'][0]['input_ids'])

'naksir bgt kulot<extra_id_0> Shopee tp mau beli mikir kali karena<extra_id_1> ngestalk lagi.<extra_id_2> gue beneran naksir ama kulotnya</s><extra_id_3> ga lagi deh pake<extra_id_4> express, kapok :")</s><extra_id_5> emang set<extra_id_6> xpress aja.. tapi setidaknya aku pake yang standard huuuu orang<extra_id_7> free bodoh<extra_id_8> ga cek</s> <unk> gapake bukalapak kakk coba wa<extra_id_9></s> <unk>Langit Merah Jakarta,'

In [12]:
tokenizer.decode(downsampled_dataset['train'][0]['labels'])

'<extra_id_0> di<extra_id_1> takut dianggap<extra_id_2> padahal<extra_id_3></s> <unk>udahlah<extra_id_4> shopee<extra_id_5></s> <unk>tapi sellernya<extra_id_6> shopee<extra_id_7> sama"<extra_id_8> banget<extra_id_9></s> aja</s><extra_id_10>'

# Model training


## Using MLM task

In [13]:
from transformers import (
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)

In [14]:
#model_checkpoint = "Wikidepia/IndoT5-base"
model = T5ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT).to("cuda")

In [15]:
datacollator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [16]:
batch_size = 8
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
#model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=10,
    save_total_limit=2,
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    tokenizer=tokenizer,
    data_collator=datacollator,
    
)

# Eval

In [18]:
import math

perplexity before training

In [19]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


100%|██████████| 245/245 [00:15<00:00, 15.34it/s]

>>> Perplexity: 92.12


In [20]:
trainer.train()

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 500/22020 [02:12<1:35:09,  3.77it/s]

{'loss': 3.2899, 'learning_rate': 1.954586739327884e-05, 'epoch': 0.23}


  5%|▍         | 1000/22020 [04:24<1:33:08,  3.76it/s]

{'loss': 2.7823, 'learning_rate': 1.9091734786557677e-05, 'epoch': 0.45}


  7%|▋         | 1500/22020 [06:34<1:27:14,  3.92it/s]

...

                                                       
100%|██████████| 22020/22020 [1:34:54<00:00,  4.80it/s]

{'eval_loss': 1.8925002813339233, 'eval_runtime': 14.1359, 'eval_samples_per_second': 138.442, 'eval_steps_per_second': 17.332, 'epoch': 10.0}


100%|██████████| 22020/22020 [1:35:14<00:00,  3.85it/s]

{'train_runtime': 5714.9929, 'train_samples_per_second': 30.812, 'train_steps_per_second': 3.853, 'train_loss': 2.0988503976695436, 'epoch': 10.0}


TrainOutput(global_step=22020, training_loss=2.0988503976695436, metrics={'train_runtime': 5714.9929, 'train_samples_per_second': 30.812, 'train_steps_per_second': 3.853, 'train_loss': 2.0988503976695436, 'epoch': 10.0})

perplexity after training

In [21]:
model.save_pretrained(SAVE_PATH)

In [22]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

100%|██████████| 245/245 [00:14<00:00, 17.46it/s]

>>> Perplexity: 6.64


In [24]:
# eval_results = trainer.evaluate()
# print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

100%|██████████| 245/245 [00:14<00:00, 17.18it/s]

>>> Perplexity: 6.46


# For BART

In [18]:
from transformers import PreTrainedTokenizerFast

In [39]:
raw_dataset = load_dataset('csv', data_files='data/quadruplet_only.csv')
tokenized_datasets = raw_dataset.map(
    tokenize_function, batched=True, remove_columns=raw_dataset['train'].column_names
)
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
downsampled_dataset = lm_datasets["train"].train_test_split(test_size=0.1, seed=42)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-5ee63b2ab57cbf46/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 500.04it/s]


In [41]:
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 72
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8
    })
})

In [53]:
tokenizer.decode(downsampled_dataset['train'][1]['input_ids'], skip_special_tokens=False)

' beli barang di dengan pengiriman pake si driver ngeluh hanya dapat perak sya sich gak percaya, masa sich sya jawab gitu, lah wong sy bayar rb sameday dari toko didepok, gk tau dah. intinya ada dua driver gosend, dan setelah gue liat kejadiannya di cctv kantor.. asumsi gue itu tu komplotan driver. pas baca thread ini, ya kok yakinnya malah penjualnya yang main. wkwkwk damn, iya bro. kitanya sebagai buyer jadi rugi. dia enak, tinggal ajuin klaim hilang terus dapet dana kita. sedangkan kita, harus nunggu approval dulu dan proses klaim segala macem. sial emang. gk mau buruk sangka'

In [52]:
tokenizer.decode(downsampled_dataset['train'][1]['input_ids'], skip_special_tokens=False).split('.')

[' beli barang di dengan pengiriman pake si driver ngeluh hanya dapat perak sya sich gak percaya, masa sich sya jawab gitu, lah wong sy bayar rb sameday dari toko didepok, gk tau dah',
 ' intinya ada dua driver gosend, dan setelah gue liat kejadiannya di cctv kantor',
 '',
 ' asumsi gue itu tu komplotan driver',
 ' pas baca thread ini, ya kok yakinnya malah penjualnya yang main',
 ' wkwkwk damn, iya bro',
 ' kitanya sebagai buyer jadi rugi',
 ' dia enak, tinggal ajuin klaim hilang terus dapet dana kita',
 ' sedangkan kita, harus nunggu approval dulu dan proses klaim segala macem',
 ' sial emang',
 ' gk mau buruk sangka']

In [ ]:
wwm_probability = 0.2


def text_infilling(examples):
    new_inputs = []
    new_labels = []
    for i in range(len(examples['input_ids'])):
        word_ids = examples['word_ids'][i]
        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = examples["input_ids"][i]
        labels = examples["labels"][i]
        new_inputs = []
        new_labels = []
        for word_id, masked in enumerate(mask):
            for idx in mapping[word_id]:
                masked_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        #dropping the same extra_id
        input_ids = drop_duplicate_id(input_ids, input_masked+1)
        labels = drop_duplicate_id(labels, label_masked+1)
        new_inputs.append(input_ids)
        new_labels.append(labels)
    examples['labels'] = new_labels
    examples['input_ids'] = new_inputs
    examples['attention_mask'] = [[1 for i in range(len(examples['input_ids'][j]))] for j in range(len(examples['input_ids']))]
    return examples

In [ ]:
# wwm_probability = 0.2


# def whole_word_masking(examples):
#     new_inputs = []
#     new_labels = []
#     for i in range(len(examples['input_ids'])):
#         word_ids = examples['word_ids'][i]
#         # Create a map between words and corresponding token indices
#         mapping = collections.defaultdict(list)
#         current_word_index = -1
#         current_word = None
#         for idx, word_id in enumerate(word_ids):
#             if word_id is not None:
#                 if word_id != current_word:
#                     current_word = word_id
#                     current_word_index += 1
#                 mapping[current_word_index].append(idx)

#         # Randomly mask words
#         mask = np.random.binomial(1, wwm_probability, (len(mapping),))
#         input_ids = examples["input_ids"][i]
#         labels = examples["labels"][i]
#         input_masked = 0
#         label_masked = 0
#         for word_id, masked in enumerate(mask):
#             for idx in mapping[word_id]:
#                 #if not masking input_ids then we mask the label
#                 if masked==0:
#                     labels[idx] = tokenizer.convert_tokens_to_ids(f'<extra_id_{label_masked}>')
#                 #if masking then we mask the input_ids
#                 elif masked==1:
#                     input_ids[idx] = tokenizer.convert_tokens_to_ids(f'<extra_id_{input_masked}>')
#             if masked==0:
#                 label_masked+=1
#             elif masked==1:
#                 input_masked+=1
#         new_inputs.append(input_ids)
#         new_labels.append(labels)
#     examples['labels'] = new_labels
#     examples['input_ids'] = new_inputs
#     return examples